In [6]:
### Sample size 100
## Sine squared and pure noise

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from tqdm import trange

# Pure guassian noise with mean = 0 stdev = 1 
def generate_data(size=25, noise=0.1):
    x = np.random.rand(size)
    x.sort()
    noise = np.random.normal(0, 1, size)
    y = noise 
    return x, y

# Sine squared + additive noise of standard devitaion of 0.2
def generate_sine_data(size=25):
    x = np.random.rand(size)
    # x.sort()
    noise = np.random.normal(0, 0.2, size)
    y = (np.sin(2 * np.pi * x)) ** 2 + noise
    return x, y

def fpe(n, DoF, Remp):
    p = DoF / n
    return Remp * ((1 + p) / (1 - p))

def gcv(n, DoF, Remp):
    p = DoF / n
    return ((1 - p) ** -2) * Remp

def sms(n, DoF, Remp):
    p = DoF / n
    return (1 + 2 * p) * Remp

def vc(n, DoF, Remp):
    p = DoF / n
    r = 1 - np.sqrt(p - p * np.log(p) + np.log(n) / (2 * n))
    if r < 0:
        return 0
    return Remp / r


def trig_func(x, m):
    if m == 0:
        Xtri = np.column_stack([np.ones(x.shape)]) # random?
        return Xtri
    Xtri = np.column_stack([(np.cos(i * x) + np.sin(i * x)) for i in range(1, m + 1)])
    Xtri = np.column_stack([np.ones(x.shape), Xtri])
    
    return Xtri

def train(x_train, y_train, x_test, y_test, degrees, eval_func=None):
    size = len(x_train)
    eval_results = []
    for m in range(degrees + 1):
        model = LinearRegression()
        X_tri = trig_func(x_train,m)
        
        eval_result = 0
        if eval_func == None:
            result = cross_val_score(
                model, X_tri, y_train, cv=size, scoring="neg_mean_squared_error"
            )
            eval_result = (result.mean())
        else:
            model.fit(X_tri, y_train)
            y_pred = model.predict(X_tri)
            mse = np.mean((y_train - y_pred) ** 2)
            eval_result = eval_func(size, m + 1, mse)

        eval_results.append(eval_result)

    opt_m = np.argmin(eval_results) + 1
    model = LinearRegression()
    X_tri_train = trig_func(x_train, opt_m)
    X_tri_test = trig_func(x_train, opt_m)
    model.fit(X_tri_train, y_train)
    y_pred = model.predict(X_tri_test)
    opt_m_mse = np.mean((y_test - y_pred) ** 2)
    
    return opt_m, opt_m_mse

def trainer(datasize, epochs, max_deg, generate_data=generate_data):
    # m for degree, mse for risk
    fpe_ms, fpe_mses = [],[]
    gcv_ms, gcv_mses = [],[]
    vc_ms, vc_mses = [],[]
    cv_ms, cv_mses = [],[]
    
    for i in trange(epochs):
        x_train, y_train = generate_data(size=datasize)
        x_test, y_test = generate_data(size=datasize)

        fpe_m, fpe_mse = train(x_train,y_train,x_test,y_test,max_deg,fpe)
        fpe_ms.append(fpe_m)
        fpe_mses.append(fpe_mse)

        gcv_m, gcv_mse = train(x_train,y_train,x_test,y_test,max_deg,gcv)
        gcv_ms.append(gcv_m)
        gcv_mses.append(gcv_mse)

        vc_m, vc_mse= train(x_train,y_train,x_test,y_test,max_deg,vc)
        vc_ms.append(vc_m)
        vc_mses.append(vc_mse)

        cv_m, cv_mse= train(x_train,y_train,x_test,y_test,max_deg,eval_func=None)
        cv_ms.append(cv_m)
        cv_mses.append(cv_mse)

    best_ms = [fpe_ms, gcv_ms, vc_ms, cv_ms]
    best_mses = [fpe_mses, gcv_mses, vc_mses, cv_mses]
    return best_ms, best_mses
    # print(fpe_ms, fpe_mses)



if __name__ == "__main__":

    best_degrees, best_risks = trainer(
        datasize=100, epochs=300, max_deg=20, generate_data=generate_data
    )

    # best_degrees, best_risks = trainer(
    #     datasize=100, epochs=300, max_deg=20, generate_data=generate_data
    # )



    plt.boxplot(best_degrees, labels=["fpe", "gev", "vc", "cv"], sym="")
    plt.title("Gussian Noise Data set wtih σ=1")
    plt.ylabel("Degree of Freedom(m)")
    plt.savefig("./img/Pure1_Degree.png")
    plt.clf()

    plt.boxplot(best_risks, labels=["fpe", "gev", "vc", "cv"], sym="")
    plt.yscale("log")
    # plt.yticks([1e10, 1, 1e-10], [r'$10^{10}$', r'$1$', r'$10^{-10}$'])
    plt.title("Gussian Noise Data set wtih σ=1")
    plt.ylabel("Risk(MSE)")
    plt.savefig("./img/Pure1_Risk(MSE).png")
    plt.clf()


    best_degrees, best_risks = trainer(
        datasize=100, epochs=300, max_deg=20, generate_data=generate_sine_data
    )

    plt.boxplot(best_degrees, labels=["fpe", "gev", "vc", "cv"], sym="")
    plt.title("Sine-squared Data set wtih noise(σ=0.2)")
    plt.ylabel("Degree of Freedom(m)")
    plt.savefig("./img/Sine_Degree.png")
    plt.clf()

    plt.boxplot(best_risks, labels=["fpe", "gev", "vc", "cv"], sym="")
    plt.yscale("log")
    # plt.yticks([1e10, 1, 1e-10], [r'$10^{10}$', r'$1$', r'$10^{-10}$'])
    plt.title("Sine-squared Data set wtih noise(σ=0.2)")
    plt.ylabel("Risk(MSE)")
    plt.savefig("./img/Sine_Risk(MSE).png")
    plt.clf()
# print(x)

# 


100%|██████████| 300/300 [04:05<00:00,  1.22it/s]
/var/folders/81/qgbbmj9x18352k83sb6q07_c0000gn/T/ipykernel_45000/70213732.py:132: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(best_degrees, labels=["fpe", "gev", "vc", "cv"], sym="")
/var/folders/81/qgbbmj9x18352k83sb6q07_c0000gn/T/ipykernel_45000/70213732.py:138: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(best_risks, labels=["fpe", "gev", "vc", "cv"], sym="")
100%|██████████| 300/300 [04:06<00:00,  1.22it/s]
/var/folders/81/qgbbmj9x18352k83sb6q07_c0000gn/T/ipykernel_45000/70213732.py:151: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.b

<Figure size 640x480 with 0 Axes>